In [ ]:
from xai.genome import Genome
from xai.bytes import GigaBytes
from xai.policy import Policy
from xai.asteroids import Asteroids
import matplotlib.pyplot as plt
import torch

In [ ]:
genome = Genome(translate=True, rotate=True, volatility=1.0)

population = genome.populate(
    population_size=25,
    max_memory=GigaBytes(5.0),
    verbose=True
)

In [ ]:
population.evolve(
    generations=10,
    survivors_cnt=1,
    elites_cnt=0,
    roulettes_cnt=0,
    random_cnt=0,
    rank_cnt=10,
    checkpoints_directory="Test",
    number_of_processes=4
)

In [ ]:
population._observations

In [ ]:
torch.cuda.empty_cache()

In [ ]:
hidden_dims = [2**10,2**7]
big_encoder = Policy.new(Asteroids.observation_shape, 100, hidden_layers=hidden_dims)
big_decoder = Policy.new(100, Asteroids.observation_shape ,hidden_layers=hidden_dims[::-1])
big_autoencoder = big_encoder + big_decoder
big_autoencoder

In [ ]:
tensor = torch.stack(population._observations.randoms(False).take(3000).map(lambda obs: obs.translated().rotated().tensor(normalize=True, device="cuda:0")).tuple()).cuda()


In [ ]:
del tensor

In [ ]:
f = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), stride=(2,2)),
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,2), stride=(1,1))
)


big_autoencoder.adam().fit(
    X=tensor,
    Y=tensor,
    epochs=5000,
    batch_size=256,
    loss_criterion="MSELoss",
    verbose=True
).plot_loss()

In [ ]:
big_autoencoder.predict(tensor[0]).tensor(True).cpu().max()

In [ ]:

idx = 2649



plt.imshow(torch.hstack([tensor[idx].cpu(), big_autoencoder.predict(tensor[idx]).tensor(True).cpu()]))